In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import rie_estimator as rie
import bahc

In [17]:
us_equities = pd.read_parquet('../data/us_equities_logreturns_cut.parquet')
display(us_equities)

,^GSPC,GE,IBM,DIS,BA,CAT,AA,HPQ,DD,KO,...,NSM,CLP,CTX,CTR,DYN,AIB,KIM,SFN,TCO,S
12002,-0.007032,-0.014556,0.010490,-0.026062,-0.022809,0.003367,-0.014736,-0.012446,-0.006903,-0.024048,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12003,0.012328,0.018882,-0.002737,0.018079,0.023845,0.007257,-0.018387,0.013397,0.003043,0.016436,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12004,-0.004491,0.002156,-0.015313,0.000943,-0.028630,-0.009500,0.019061,-0.026488,-0.013068,0.008623,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12005,0.027738,0.032489,0.034315,0.023739,0.015598,0.040170,0.027927,0.039239,0.003911,0.029363,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12006,-0.002796,0.016541,0.000000,-0.002303,0.018969,-0.007037,-0.015197,-0.012751,0.022875,-0.003439,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16286,0.000305,-0.005420,-0.002896,0.004063,-0.009916,-0.000207,-0.001642,-0.010756,0.006343,-0.000520,...,0.042972,NaN,0.001974,-0.001879,0.011547,0.003892,0.002334,0.011191,0.009188,0.000000
16287,-0.011770,-0.016439,-0.012537,-0.017978,-0.020624,-0.012576,-0.025803,-0.023249,-0.010898,-0.013082,...,-0.011131,NaN,0.007076,-0.006604,-0.008647,-0.001296,0.001863,0.001534,-0.000436,-0.018391
16288,-0.002826,-0.009118,-0.009232,0.001750,0.009959,-0.002304,-0.001688,0.005519,0.000913,0.005253,...,0.047769,NaN,0.007026,0.003307,0.008237,0.003020,0.001395,0.005350,-0.015368,0.018391
16289,0.009568,0.012663,0.019565,0.016351,0.018590,0.002931,0.005055,0.014005,0.009232,0.002616,...,-0.017531,NaN,0.003882,0.007987,0.002458,0.025942,0.000465,0.009860,0.000442,-0.001140


In [30]:
us_equities_filtered = us_equities.loc[15500:].dropna(axis=1)
# Calibration Window
T = np.arange(15501, 16001, 1)

calibration_us_equities = us_equities_filtered.loc[15500:16000].iloc[:,:100]

# Number of Assets
N = calibration_us_equities.shape[1]

print(f'Number of Assets: {N}')
print(f'Number of Days: {len(T)}')

cov = calibration_us_equities.cov()
inv_cov = np.linalg.inv(cov)
e = np.ones(N)

gmvp_weights = inv_cov.dot(e) / (e.T.dot(inv_cov).dot(e))


Number of Assets: 100
Number of Days: 500


In [32]:
test_us_equities = us_equities_filtered.loc[16001:].iloc[:,:100]
cov_out = test_us_equities.cov()
realized_risk = gmvp_weights.T.dot(cov_out).dot(gmvp_weights)
print(f'Realized Risk: {realized_risk}')

Realized Risk: 4.596505597340273e-05


In [33]:
def eigenvalue_clip(data):
    T, N = data.shape
    eig, eig_vec = np.linalg.eig(data)
    lambda_p = (1 + np.sqrt(N/T))**2
    clip = eig > lambda_p
    N_clip = N - np.sum(clip)
    sum_lambda = np.sum(eig[clip])
    delta = (N - sum_lambda)/N_clip
    eig[np.logical_not(clip)] = delta
    C = eig_vec.T @ np.diag(eig) @ eig_vec
    C = C - np.diag(np.diag(C)) + np.eye(N)
    return C

In [37]:
clipped_cov = eigenvalue_clip(calibration_us_equities.cov())
rie_cov = rie.get_rie(calibration_us_equities, normalize=False)
bahc_cov = bahc.filterCovariance(calibration_us_equities.values)

e_val, e_vec = np.linalg.eig(calibration_us_equities.cov())
ao_e_val = pd.read_csv('../data/10_Average_Oracle_eigenvalues_N=100.csv').values
ao_cov = e_vec.T.dot(ao_e_val).dot(e_vec)

ValueError: shapes (100,2) and (100,100) not aligned: 2 (dim 1) != 100 (dim 0)